In [1]:
# valuation_date時におけるcodeで指定した銘柄の個別銘柄分析
# SBI証券のモバイルの銘柄分析を参考に作成

In [2]:
### パラメータ設定セル
from datetime import date
from dateutil.relativedelta import relativedelta

# パラメータをセット
# code: 証券コード
# valuation_date: 過去にさかのぼって評価日をdatetime.dateでセット可能。デフォルト値は今日(date.today())
code = 4026
valuation_date = date.today()
#valuation_date = date(2024, 12, 13)


In [3]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import PricelistPl, FinancequotePl, KessanPl, KessanFig, PricelistFig, read_data, get_df_latest_yearly_performance, get_latest_stockprice, print_finance_quote
from lib_dataprocess import get_fig_expected_performance_progress_rate_pycharts, MeigaralistPl, get_fig_actual_performance_progress_rate_pycharts, ShikihoPl
import polars as pl

# dataの読み込み
fp = DATA_DIR/"raw_pricelist.parquet"
df = read_data(fp)
PPL = PricelistPl(df)

# dataの読み込み
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RevPPL = PricelistPl(df)

fp = DATA_DIR/"finance_quote.parquet"
df = read_data(fp)
FPL = FinancequotePl(df)
FPL.with_columns_market_cap()

fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL = KessanPl(df)

fp = DATA_DIR/"meigaralist.parquet"
df = read_data(fp)
MPL = MeigaralistPl(df)

fp = DATA_DIR/"shikiho.parquet"
df = read_data(fp)
SPL = ShikihoPl(df)

In [4]:
### 最新情報
# 指定したcodeの指定した日における株価と各種ファンダメンタルズデータをまとめて標準出力する
# pricelist_dfは、raw_pricelistかreviced_pricelistかケースに応じて使い分ける。
# def print_finance_quote(
#        pricelist_df: pl.DataFrame,
#        finance_quote_df: pl.DataFrame,
#        code: int, 
#        valuation_date: date=date.today()
#    ) -> None:
print_finance_quote(PPL.df, FPL.df, code, valuation_date=valuation_date)
print()
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)

神島化学工業(4026)の銘柄情報

終値: 1536.0円(2024年12月26日)
予想配当利回り: 2.86%(2024年12月26日)
予想PER: 8.43倍(2024年12月26日)
実績PBR: 1.13倍(2024年12月26日)
自己資本比率: 39.8%(2024年12月26日)
予想ROE: 13.36%(2024年12月26日)
予想ROA: 5.32%(2024年12月26日)
時価総額: 141.9億円(2024年12月26日)

4026(神島化学工業)の四季報データ履歴

発行日: 2024年09月13日
【上向く】
  サプリや自動車向けマグネシウム、車載用やレーザー向けセラミックスが伸長し業績牽引。建材も高級軒天ボード採用拡大など高付加価値品下支え。減価償却費など固定費増吸収し増益転換。連続増配。
【核融合発電】
  ITER計画(トカマク方式)が大幅遅延の公算、レーザー核融合方式の市場浸透チャンスに。建材は販売チャネル強化によって非住宅向け拡販進める。

発行日: 2024年06月14日
【改善】
  25年4月期は建材横ばい見込みも、セラミックスが車載用蛍光体の拡大持続やレーザー核融合発電向け寄与で伸長。収益性高い酸化マグネシウムも拡販進展。償却負担大幅増の影響も一巡し営業増益。
【新分野】
  CO2固定化建材製造に向けた大型投資は25年4月期後半に予定、翌期に販売開始本格化。プロジェクター用途『LED用蛍光体』は試験用向けに販売開始。

発行日: 2024年03月16日
【堅調】
  マグネシウムはじめ化成品が設備増強効果で増販進む。建材は高級軒天ボードなど高付加価値品が下支え。修繕費などの一部固定費負担が後半に低減。増配。25年4月期は化成品を牽引役に営業増益。
【協調】
  大阪大学レーザー科学研究所とセラミックス協働拠点設立、核融合発電分野の開発強化。25年にCO2固定化建材の製造開始予定、25年4月期設備投資実施も。

発行日: 2023年12月15日
【連続最高益】
  化成品が増産設備稼働で伸長、利益率高いサプリ向けマグネシウムも在庫調整解消し後半巻き返し。建材は高付加価値品が好調で下支え。減価償却増えるが、増収効果でかわす。営業益堅調。増配。
【セラミックス】
  プロジェクター用途のLED用蛍光体サンプル出

In [5]:
### 業績推移

# KessanFig(
#    code: int, 
#    settlement_type: Literal["通期", "四半期"], 
#    output_target: str = "jupyter",
#    start_settlement_date: date = date(1900, 1, 1),
#    end_settlement_date: date = date(2999, 12, 31)
#)

KFIG = KessanFig(code, "通期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [6]:
# 今季決算予想発表の推移
df = KPL.get_settlement_forcast(code, valuation_date)
df

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
4026,2025-04-30,"""予""",2024-06-12,27400,2350,2300,1650,182.3,44.0,4
4026,2024-04-30,"""予""",2023-06-12,25700,2200,2200,1600,177.0,42.0,4


In [7]:
### 通期業績、業績予想
# def get_df_latest_yearly_performance(code: int, valuation_date: date=date.today()) -> pl.DataFrame:
df = get_df_latest_yearly_performance(code, valuation_date)
df

神島化学工業(4026)の通期決算(評価日：2024年12月27日)


決算期,決算発表日,売上高,営業利益,経常利益,純利益,EPS,1株配当
str,date,i64,i64,i64,i64,f64,f64
"""2025年4月期(予)""",2024-06-12,27400,2350,2300,1650,182.3,44.0
"""2024年4月期""",2024-06-12,25974,2117,2073,1620,179.1,42.0


In [8]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()
KPL.get_latest_quater_settlement(code, valuation_date)

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
4026,2024-10-31,"""四""",2024-12-11,6921,369,349,253,28.0,5.3,2
4026,2024-07-31,"""四""",2024-09-11,6983,649,633,453,50.1,9.3,1
4026,2024-04-30,"""四""",2024-06-12,6822,799,788,648,71.6,11.7,4
4026,2024-01-31,"""四""",2024-03-12,6540,425,410,318,35.2,6.5,3
4026,2023-10-31,"""四""",2023-12-12,6417,478,465,381,42.2,7.4,2
4026,2023-07-31,"""四""",2023-09-12,6195,415,410,273,30.2,6.7,1
4026,2023-04-30,"""四""",2023-06-12,6349,523,516,512,56.7,8.2,4
4026,2023-01-31,"""四""",2023-03-10,6159,584,574,400,44.3,9.5,3


In [9]:
# codeで指定した銘柄のevaluation_dateで指定した時点での最新の年度決算予想に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# from plotly.graph_objects import Figure
# def get_fig_expected_performance_progress_rate_pycharts(code: int, evaluation_date: date=date.today()) -> Figure:

fig1 = get_fig_expected_performance_progress_rate_pycharts(code, valuation_date)
fig1.show()

神島化学工業(4026)の2025年4月期第2四半期決算進捗率(評価日：2024-12-27)


In [10]:
last_year_valuation_date = date(valuation_date.year-1, valuation_date.month, valuation_date.day)

# codeで指定した銘柄のevaluation_dateで指定した時点での最新の四半期決算発表に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# valuation_dateで指定する日において、当年度の本決算が発表されていない日を指定した場合は前期末第4四半期の進捗率を表示するので、進捗率100%で表示される。
fig2 = get_fig_actual_performance_progress_rate_pycharts(code, last_year_valuation_date, KPL.df, MPL.df)
fig2.show()

神島化学工業(4026)の2024年4月期第2四半期決算進捗率(評価日：2023-12-27)


In [11]:
#### valuation_dateに過去日を指定した場合、valuation_dateを含む期の株価推移
#### 答え合わせ

In [12]:
# 株価推移
# 前期算発表翌営業日～今期決算発表日翌営業日
start_date, end_date = KPL.get_current_settlement_period_by_announcement_date(code, valuation_date, "四")

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>start_date)
start_date = df["date"].to_list()[0]

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>end_date)
if not end_date == date(2999, 12, 31):
    end_date = df["date"].to_list()[0]

PFIG = PricelistFig(code, RevPPL.df, MPL.df, start_date, end_date)
df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")<=valuation_date)
vline_date = df["date"].to_list()[-1]
PFIG.add_vline(vline_date)
PFIG.fig.show()

In [13]:
# appendix
KPL.df.filter(pl.col("code")==code).filter(pl.col("announcement_date")>=date(2024,1,1))

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
4026,2024-01-31,"""四""",2024-03-12,6540,425,410,318,35.2,6.5,3
4026,2024-04-30,"""四""",2024-06-12,6822,799,788,648,71.6,11.7,4
4026,2024-04-30,"""本""",2024-06-12,25974,2117,2073,1620,179.1,42.0,4
4026,2025-04-30,"""予""",2024-06-12,27400,2350,2300,1650,182.3,44.0,4
4026,2024-07-31,"""四""",2024-09-11,6983,649,633,453,50.1,9.3,1
4026,2024-10-31,"""四""",2024-12-11,6921,369,349,253,28.0,5.3,2


In [14]:
PFIG = PricelistFig(code, RevPPL.df, MPL.df, date(2023, 12, 1), date.today())
PFIG.fig.show()

In [15]:
# 次の決算どうだったか
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date+relativedelta(days=100))
KFIG.add_trace_profits()
KFIG.fig.show()

In [16]:
from lib_dataprocess import CreditbalancePl

CPL = CreditbalancePl()
CPL.with_columns_margin_ratio()
CPL.df.filter(pl.col("code")==code)

code,date,unsold_margin,purchase_margin,margin_ratio
i64,date,i64,i64,f64
4026,2008-12-30,0,3000,inf
4026,2009-01-09,0,3000,inf
4026,2009-01-16,0,3000,inf
4026,2009-01-23,0,7000,inf
4026,2009-01-30,0,3000,inf
…,…,…,…,…
4026,2024-11-15,22100,376900,17.05
4026,2024-11-22,21800,378200,17.35
4026,2024-11-29,21400,386700,18.07
